# Write-up notes

## Multi-class logistic regression

## Algorithims
### Gradient Descent
### BCGD
What is the motivation of BCGD - I think it is exploiting convexity to make less work.  
Random rule - does not require calculating the full gradent, but only one block at at time. This saves work at each iteration, but can waste time selecting blocks that have already been optimized.  
Optimality conditions: check norm of gradient or block of gradient. if less than 0.01 stop. For random rule we use paticence, since selecting previously optimized blocks will have low gradient and would otherwise trigger stopping before convergence.  
Formally state the steps of each algorithim.  
Will probally discuss the results in the dataset sections.

## Step-size
The more theoretical formulations for block coordinate descent methods use line searching to find an optimal point in the direction of the current gradient block. This does well to minimize the number of iterations for convergence by exploiting the convexity of the logistic classification problem. [expand]
Here we visualize the loss along a gradient block selected at an iteration of BCGD with GS rule with varying step sizes.
[image]
In practice computing he loss at many different points along this line to find the optimal step is a time expensive operation. Second order methods 
Calculating the Lipchitz constant... Requires hessian. This is expensive. I think this would need to be done at each iteration, too. Can use approximation of lipshitz constant to create fixed stepsize that works well enough. Maybe mention adaptive rates. Reference slides for writing this section.

## Syntetic Dataset
Gradient descent performs noticably better than both BCGD methods. We attribute this to the high-dimensionality of examples resulting in a large number of blocks that need to be individually optimized, where methods using the full gradient can decrease many of these at once.

## Real world data